# Land Diagnostics Framework (LDF)

**NOTE** LDF compares two cases and currently has limited observations available in regional climatology plots.  It works when comparing two structured (e.g. f09) or unstructured (ne30) cases.  No regridding is required for comparisons of unstructured cases.

[Sample output from LDF is in this link](https://webext.cgd.ucar.edu/I20TR/ctsm5.4.004_bgc_f09_131_HIST/lnd/computed_notebooks/_build/html/infrastructure/index.html).

LDF can be run in CUPiD via the following process:
#### 1) Install CUPiD, submodues (including LDF) and conda environments
   * `git clone --recurse-submodules https://github.com/NCAR/CUPiD.git`
   * `cd CUPiD`
   * set up conda environments [cupid-analysis, cupid-infrastructure]
   * `conda activate cupid-infrastructure`

#### 2) Set up the CUPiD configuration file

   * `cd CUPiD/examples/land_only`

_This example will create a subset of LDF output for quicker analyses_
  
   * `cp config_subset.yml config.yml`

_OR for the full LDF output on a BGC case:_

   * `cp config_full.yml config.yml`

Then modify the `config.yml` file to point to the cases you want to run

#### 3) Generate LDF config files based on a CUPiD configuration file.
This step uses the `CUPiD/helper_scripts/generate_ldf_config_file.py` 

_This example will run LDF on structured cases (f09 grid)_

   * `../../helper_scripts/generate_ldf_config_file.py --cupid-config-loc . --ldf-template ../../externals/LDF/config_clm_structured_plots.yaml --out-file ldf_config.yml`
 
_This example will run LDF on unstructured cases (ne30 grid)_

   * `../../helper_scripts/generate_ldf_config_file.py --cupid-config-loc . --ldf-template ../../externals/LDF/config_clm_unstructured_plots.yaml --out-file ldf_config.yml`


#### 4) Launch an interactive job on casper
   * `export PBS_ACCOUNT=P#########` (e.g. P93300041 for the LMWG)
   * `qinteractive -l select=1:ncpus=16:mpiprocs=16:mem=100G -q casper@casper-pbs -A PBS_ACCOUNT -l walltime=08:00:00 `

#### 5) Run LDF with the newly created configuration file.
**NOTE 1:** If an LDF_output dir already exists in CUPiD/examples/land_only, remove it
**NOTE 2:** LDF will generate it's own single variable timeseries and climatology plots in your scratch directory.  If you've already run LDF for a case with the same time periods as you're analizing here it will reuse those files.  If you're running over a different time period here, however, LDF will get confused by the old files so it's better to remover them

   * `conda activate cupid-analysis`
   * `module load nco`
   * `../../externals/LDF/run_adf_diag ldf_config.yml`

#### 6) After LDF has finished, run CUPiD diagnostics & webpage

   * `conda activate cupid-infrastructure`
   * `cupid-diagnostics -lnd`
   * `cupid-webpage`

**Note:** you can always remove a computed notebook & re-run cupid-diagnostics to get an updated version, e.g. if you run this before the LDF output exists and you want to rerun ILAMB later

#### 6) View webpage:
  Can use another method, but it’s simple to scp the computed notebook directory locally and then open the index.html page


------
Code below was posted previously, but may not be relevent since LDF is now managed as a submodule to CUPiD?

In [ ]:
import os

from IPython.core.display import HTML, Image
from IPython.display import display
import pandas as pd

In [ ]:
ldf_root = "."
case_name = None
base_case_name = None
start_date = ""
end_date = ""
base_start_date = None
base_end_date = None
key_plots = None
# ldf_root will be external_diag_packages/computed_notebooks/LDF/

In [ ]:
# Want some base case parameter defaults to equal control case values
if base_case_name is not None:
    if base_start_date is None:
        base_start_date = start_date

    if base_end_date is None:
        base_end_date = end_date

# convert start-date and end-date to year range
case_year_range = [int(start_date.split("-")[0]), int(end_date.split("-")[0])]

if (not base_start_date) and (not base_end_date):
    base_case_year_range = None
else:
    base_case_year_range = [
        int(base_start_date.split("-")[0]),
        int(base_end_date.split("-")[0]),
    ]

In [ ]:
if base_case_year_range:
    base_case_yr_range_str = f"_{base_case_year_range[0]}_{base_case_year_range[1]}"
    alt_base_case_yr_range_str = (
        f"_{base_case_year_range[0]}_{str(int(base_case_year_range[1])-1)}"
    )
else:
    base_case_yr_range_str = ""
    alt_base_case_yr_range_str = ""


possible_ldf_comparison_names = [
    f"{case_name}_{case_year_range[0]}_{case_year_range[1]}_vs_{base_case_name}{base_case_yr_range_str}"
]
possible_ldf_comparison_names.append(
    f"{case_name}_{case_year_range[0]}_{str(int(case_year_range[1])-1)}_vs_{base_case_name}{base_case_yr_range_str}"
)
possible_ldf_comparison_names.append(
    f"{case_name}_{case_year_range[0]}_{str(int(case_year_range[1])-1)}_vs_{base_case_name}{alt_base_case_yr_range_str}"
)
possible_ldf_comparison_names.append(
    f"{case_name}_{case_year_range[0]}_{case_year_range[1]}_vs_{base_case_name}{alt_base_case_yr_range_str}"
)

In [ ]:
ldf_found = False
for ldf_comparison_name in possible_ldf_comparison_names:
    ldf_root_candidate = os.path.join(ldf_root, ldf_comparison_name)
    if os.path.exists(ldf_root_candidate):
        ldf_found = True
        ldf_root = ldf_root_candidate
        display(
            HTML(
                f'<a href="../LDF/{ldf_comparison_name}/website/index.html" target="_blank" style="font-size: 30px">Full LDF output</a>'
            )
        )
        break

if not ldf_found:
    print("No LDF output found for the specified case and date range.")

## Key Metrics from LDF

Some important things to look at from LDF include a comparison table and a few maps:

In [ ]:
comparison_table = os.path.join(ldf_root, "amwg_table_comp.csv")
if os.path.isfile(comparison_table):
    table = pd.read_csv(comparison_table)
    display(HTML(table.to_html(index=False, float_format="{:6g}".format)))

In [ ]:
for path_to_key_plot in key_plots:
    full_path = os.path.join(ldf_root, path_to_key_plot)
    if os.path.isfile(full_path):
        display(Image(full_path))